# Deutsche's Algorithm

*By Chayapol "Due" Hongsrimuang, G00388741*

---

## Introduction
Deutsche's algorithm is ....

# Operations (or Gates)
Deutsche's algorithm involves a number of unitary (...) operations to be applied for each bit passing through.

## Hadamard Operation

Hadamard operation (or Hadamard gate) is a unitary operation on a qubit, represented by H, and has these basic qubit states:

$$
|0\rangle = \frac{|0\rangle+|1\rangle}{\sqrt{2}}
|1\rangle = \frac{|0\rangle-|1\rangle}{\sqrt{2}}
$$

Which maps into the following matrix.

$$
\begin{pmatrix}
\frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\
\frac{1}{\sqrt{2}} & - \frac{1}{\sqrt{2}} 
\end{pmatrix}
$$

Each qubit state, when parsed through would transform the qubit between the regular 0 and 1 state with the + and - state as follow:

$$
H|0\rangle = \begin{pmatrix}
\frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\
\frac{1}{\sqrt{2}} & - \frac{1}{\sqrt{2}} 
\end{pmatrix}
\begin{pmatrix} 1 \\ 0 \end{pmatrix} = 
\begin{pmatrix}
\frac{1}{\sqrt{2}}\\
\frac{1}{\sqrt{2}}
\end{pmatrix} =
|+\rangle
$$
$$
H|1\rangle = \begin{pmatrix}
\frac{1}{\sqrt{2}} & \frac{1}{\sqrt{2}} \\
\frac{1}{\sqrt{2}} & - \frac{1}{\sqrt{2}} 
\end{pmatrix}
\begin{pmatrix} 0 \\ 1 \end{pmatrix} = 
\begin{pmatrix}
\frac{1}{\sqrt{2}}\\
- \frac{1}{\sqrt{2}}
\end{pmatrix} =
|-\rangle
$$

This operation can be denotated with this Operator.

In [1]:
import numpy as np
from qiskit.quantum_info import Operator


H = Operator([[1 / np.sqrt(2), 1 / np.sqrt(2)], [1 / np.sqrt(2), -1 / np.sqrt(2)]])